# Assignment 2: Milestone I Natural Language Processing

<h3 style="color:#ffc0cb;font-size:50px;font-family:Georgia;text-align:center;"><strong>Task 1. Basic Text Pre-processing</strong></h3>

#### Student Name: Tran Ngoc Anh Thu
#### Student ID: s3879312

Date: "October 2, 2022"

Version: 1.0

Environment: Python 3 and Jupyter notebook

Libraries used: 
* sklearn
* collections
* re
* numpy
* nltk
* itertools
* pandas
* os

## Steps
1. Load data
2. Text Pre-processing
    * Sentence Segmentation
    * Word Tokenization
    * Removing Single Character Tokens
    * Removing Stop words
3. Saving the Pre-processing Reviews

## Introduction
Nowadays there are many job hunting websites including seek.com.au and au.indeed.com. These job hunting sites all manage a job search system, where job hunters could search for relevant jobs based on keywords, salary, and categories. In previous years, the category of an advertised job was often manually entered by the advertiser (e.g., the employer). There were mistakes made for category assignment. As a result, the jobs in the wrong class did not get enough exposure to relevant candidate groups.
With advances in text analysis, automated job classification has become feasible; and sensible suggestions for job categories can then be made to potential advertisers. This can help reduce human data entry error, increase the job exposure to relevant candidates, and also improve the user experience of the job hunting site. In order to do so, we need an automated job ads classification system that helps to predict the categories of newly entered job advertisements.

In this **task1** notebook, we are going to explore a job advertisement data set, and focus on pre-processing the description only.
In the next task **task2_3**, we will then use the pre-processed text reviews to generate data features and build classification models to predict label of the description.

## Dataset
+ A small collection of job advertisement documents (around 776 jobs) inside the `data` folder.
+ Inside the data folder, there are four different sub-folders: Accounting_Finance, Engineering, Healthcare_Nursing, and Sales, representing a job category.
+ The job advertisement text documents of a particular category are in the corresponding sub-folder.
+ Each job advertisement document is a txt file named `Job_<ID>.txt`. It contains the title, the webindex (some will also have information on the company name, some might not), and the full description of the job advertisement.



## Importing libraries 

In [1]:
# import libraries
import numpy as np
import pandas as pd
from sklearn.datasets import load_files
from collections import Counter
from nltk import RegexpTokenizer
from nltk.tokenize import sent_tokenize
from itertools import chain
import re
import os

In [2]:
# check the version of the main packages
print("Numpy version: ", np.__version__)
print("Pandas version: ",pd.__version__)
! python --version

Numpy version:  1.21.5
Pandas version:  1.4.2
Python 3.9.12


<h3 style="color:#ffc0cb;font-size:50px;font-family:Georgia;text-align:center;"><strong>1.1 Examining and loading data</strong></h3>

- Examine the data folder, including the categories and job advertisment txt documents, etc. Explain your findings here, e.g., number of folders and format of txt files, etc.
- Load the data into proper data structures and get it ready for processing.
- Extract webIndex and description into proper data structures.



Before doing any pre-processing, we need to load the data into a proper format. 
To load the data, you have to explore the data folder. Inside the `data` folder:
+ Inside the data folder you will see 4 different subfolders, namely: `Accounting_Finance`, `Engineering`,`Healthcare_Nursing`, and `Sales`, each folder name is a job category.
+ The job advertisement text documents of a particular category are located in the corresponding subfolder.
+ Each job advertisement document is a txt file, named as "Job_<ID>.txt". It contains the title, the webindex,(some will also have information on the company name, some might not), and the full description of the job advertisement. 

In this case, providing that the dataset is given in a very well organised way, I would use a super handy API [`load_files`](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_files.html) from `sklearn.datasets`. 
    
**import the function by:**
```python
from sklearn.datasets import load_files  
```

Then you can use the function to directly load the data and labels, for example:
```python
df = load_files(r"data")  
```

The loaded `movie_data` is then a dictionary, with the following attributes:

| **ATTRIBUTES**   | **DESCRIPTION**                                           |
|--------------|---------------------------------------------------------------|
| Webindex     | 8 digit Id of the job advertisement on the website            |
| Title        | Title of the advertised job position                          |
| Company      | Company (employer) of the advertised job position             |
| Description  | the description of each job advertisement                     |


- Examine the data folder, including the categories and job advertisment txt documents, etc. Explain your findings here, e.g., number of folders and format of txt files, etc.
- Load the data into proper data structures and get it ready for processing.
- Extract webIndex and description into proper data structures.


In [3]:
# load each folder and file inside the data folder
df = load_files(r"data")
# type of the loaded file
type(df)

sklearn.utils.Bunch

In [4]:
df['target'] # this corresponding to the index value of the 4 categories

array([0, 0, 2, 0, 2, 1, 2, 0, 3, 3, 0, 0, 1, 3, 1, 3, 3, 1, 3, 2, 2, 2,
       3, 3, 0, 2, 2, 2, 0, 2, 3, 1, 2, 0, 1, 3, 3, 1, 1, 0, 2, 2, 2, 2,
       0, 0, 2, 1, 3, 1, 1, 2, 2, 3, 0, 0, 1, 0, 2, 2, 3, 3, 3, 0, 3, 0,
       1, 2, 3, 1, 3, 2, 3, 1, 3, 2, 1, 3, 2, 1, 3, 2, 2, 1, 0, 1, 1, 1,
       3, 0, 3, 1, 3, 2, 2, 0, 2, 3, 2, 1, 0, 1, 1, 2, 0, 3, 0, 1, 3, 2,
       1, 2, 0, 3, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 3, 1, 1, 3,
       2, 0, 0, 1, 3, 2, 0, 1, 0, 3, 1, 2, 1, 0, 0, 0, 3, 0, 1, 2, 3, 1,
       1, 1, 2, 1, 0, 1, 0, 1, 0, 1, 1, 2, 0, 2, 2, 0, 2, 3, 2, 2, 0, 2,
       1, 0, 1, 1, 1, 3, 1, 3, 1, 0, 3, 1, 0, 2, 0, 0, 2, 1, 1, 0, 1, 3,
       0, 1, 1, 3, 0, 1, 0, 2, 3, 0, 2, 0, 1, 0, 1, 3, 1, 0, 1, 1, 0, 1,
       0, 1, 2, 1, 3, 1, 2, 3, 1, 1, 2, 0, 0, 1, 2, 0, 3, 2, 3, 2, 2, 3,
       0, 1, 1, 1, 1, 1, 1, 0, 3, 1, 1, 0, 0, 2, 1, 2, 2, 2, 2, 1, 3, 1,
       2, 1, 2, 3, 2, 3, 0, 1, 3, 0, 2, 1, 0, 2, 1, 2, 0, 2, 1, 1, 1, 2,
       2, 1, 2, 0, 2, 2, 1, 2, 0, 1, 0, 0, 3, 2, 1,

In [5]:
# Name of the categories
df['target_names'] # this corresponding to the name value of the 4 categories

['Accounting_Finance', 'Engineering', 'Healthcare_Nursing', 'Sales']

In [6]:
print(f'Category at index 0: {df["target_names"][0]}')
print(f'Category at index 1: {df["target_names"][1]}')
print(f'Category at index 2: {df["target_names"][2]}')
print(f'Category at index 3: {df["target_names"][3]}')

Category at index 0: Accounting_Finance
Category at index 1: Engineering
Category at index 2: Healthcare_Nursing
Category at index 3: Sales


In [7]:
# test whether it matches, just in case
emp = 20 # an example, note we will use this example to test for the whole task outputs.

df['filenames'][emp], df['target'][emp] # from the file path we know that it's the correct label too

('data/Healthcare_Nursing/Job_00491.txt', 2)

In [8]:
# assign variables
full_description, category, directory = df.data, df.target, df.filenames

# the 20th job advertisement description
print(f'Job description: {full_description[emp]}\n\nCorresponding to the label {category[emp]} inside the {directory[emp]} directory')

Job description: b'Title: PERM Unit Mgr RGN Kid minster Flexi ****K due\nWebindex: 71692209\nDescription: Job Title: Unit Manager Reporting to: Registered Manager Job Purpose: To manage in a professional manner the day to day running of the home\xe2\x80\x99s administration, clinical policies and procedures, training and care planning. To implement working practices that monitors the health and welfare of the home\xe2\x80\x99s service users and staff and their respective environments. To promote quality care within a warm friendly ambience. Key Result Areas Managing To work with the Directors to achieve the home\xe2\x80\x99s financial targets. To manage the home in a manner which will not bring the home or service users into disrepute. To maintain confidentiality on all aspects of care and staff management. To ensure all the home\xe2\x80\x99s policies and procedures are implemented and followed by all staff. To inform the Registered Manager immediately if a serious difficulty or event o

### ------> OBSERVATION:
As we can see the current description is in the **binary** form and read as a byte object (a `b` in front of each review text if you print it out). Therefore, we need to decode into normal string for further pre-processing

However, the tokenizer cannot apply a string pattern on a bytes-like object. To resolve this, we decode each read `full_description` text using `utf-8` by writing a decode function


In [9]:
def decode(l):
    if isinstance(l, list):
        return [decode(x) for x in l]
    else:
        return l.decode('utf-8')

# decode the binary description into utf-8 form and save it to full_description
full_description = decode(full_description)
full_description[emp]

'Title: PERM Unit Mgr RGN Kid minster Flexi ****K due\nWebindex: 71692209\nDescription: Job Title: Unit Manager Reporting to: Registered Manager Job Purpose: To manage in a professional manner the day to day running of the home’s administration, clinical policies and procedures, training and care planning. To implement working practices that monitors the health and welfare of the home’s service users and staff and their respective environments. To promote quality care within a warm friendly ambience. Key Result Areas Managing To work with the Directors to achieve the home’s financial targets. To manage the home in a manner which will not bring the home or service users into disrepute. To maintain confidentiality on all aspects of care and staff management. To ensure all the home’s policies and procedures are implemented and followed by all staff. To inform the Registered Manager immediately if a serious difficulty or event occurs. Managing Support To delegate responsibility effectively

### ---------------> OBSERVATION:
The current `description` contains these attributes:

| **ATTRIBUTES**   | **MEANING**                                        |
|--------------|----------------------------------------------------|
| Webindex     | 8 digit Id of the job advertisement on the website |
| Title        | Title of the advertised job position               |
| Company      | Company (employer) of the advertised job position  |
| Description  | the description of each job advertisement          |

and I only want the description itself to perform text-preprocessing and NLP on `description`. Therefore, I will perform the following pre-processing steps to the description of each job advertisement;

<h3 style="color:#ffc0cb;font-size:50px;font-family:Georgia;text-align:center;"><strong>1.2 Pre-processing</strong></h3>

## 1.2.1 Extract information from each job advertisement
1. Extract information from each job advertisement. Perform the following pre-processing steps to the description of each job advertisement;
2. Tokenize each job advertisement description. The word tokenization must use the following regular expression, `r"[a-zA-Z]+(?:[-'][a-zA-Z]+)?"`;
3. All the words must be converted into the lower case;
4. Remove words with length less than 2.
5. Remove stopwords using the provided stop words list (i.e, stopwords_en.txt). It is located inside the
same downloaded folder.
6. Remove the word that appears only once in the document collection, based on term frequency.
7. Remove the top 50 most frequent words based on document frequency.
8. Save all job advertisement text and information in txt file(s) (you have flexibility to choose what format
you want to save the preprocessed job ads, and you will need to retrieve the pre-processed job ads
text in Task 2 & 3);
9. Build a vocabulary of the cleaned job advertisement descriptions, save it in a txt file (please refer to the
required output);

In [10]:
# Extract description, title, webindex,  from each job advertisement and test using emp

# Extract description
def extract_description(full_description):
    description = [re.search(r'\nDescription: (.*)', str(i)).group(1) for i in full_description]
    return description
description = extract_description(full_description)
print(f'Job description at index {emp}:\n{description[emp]}\n')

# Extract title
def extract_title(full_description):
    title = [re.search(r'Title: (.*)', str(i)).group(1) for i in full_description]
    return title
title = extract_title(full_description)
print(f'Job title at index {emp}:\n{title[emp]}\n')

# Extract webindex
def extract_webindex(full_description):
    webindex = [re.search(r'Webindex: (.*)', str(i)).group(1) for i in full_description]
    return webindex
webindex = extract_webindex(full_description)
print(f'Job webindex at index {emp}:\n{webindex[emp]}\n')

# Extract company
def extract_company(company):
    company = [re.search(r'Company: (.*)', str(i)).group(1) if re.search(r'Company: (.*)', str(i)) else "NA" for i in company]
    return company
company = extract_company(full_description)
print(f'Job company at index {emp}:\n{company[emp]}\n')

Job description at index 20:
Job Title: Unit Manager Reporting to: Registered Manager Job Purpose: To manage in a professional manner the day to day running of the home’s administration, clinical policies and procedures, training and care planning. To implement working practices that monitors the health and welfare of the home’s service users and staff and their respective environments. To promote quality care within a warm friendly ambience. Key Result Areas Managing To work with the Directors to achieve the home’s financial targets. To manage the home in a manner which will not bring the home or service users into disrepute. To maintain confidentiality on all aspects of care and staff management. To ensure all the home’s policies and procedures are implemented and followed by all staff. To inform the Registered Manager immediately if a serious difficulty or event occurs. Managing Support To delegate responsibility effectively and within legal boundaries. To ensure through clinical st

## 1.2.2 + 1.2.3 Tokenize each job advertisement description lowing regular expression & lowercase all words

In [11]:
def tokenizeDescription(raw_description):
    """
        This function first convert all words to lowercases,
        it then segment the raw description into sentences and tokenize each sentences
        and convert the description to a list of tokens.
    """
    description = raw_description.lower() # convert all words to lowercase

    # segment into sentences
    sentences = sent_tokenize(description)

    # tokenize each sentence
    pattern = r"[a-zA-Z]+(?:[-'][a-zA-Z]+)?"
    tokenizer = RegexpTokenizer(pattern)
    token_lists = [tokenizer.tokenize(sen) for sen in sentences]

    # merge them into a list of tokens
    tokenised_description = list(chain.from_iterable(token_lists))
    return tokenised_description

tk_description = [tokenizeDescription(r) for r in description]  # list comprehension, generate a list of tokenized articles

print("Raw description:\n",description[emp],'\n')
print("Tokenized description:\n",tk_description[emp],'\n\n')
print("The original number of Tokenized description tokens: ",len(tk_description))

Raw description:
 Job Title: Unit Manager Reporting to: Registered Manager Job Purpose: To manage in a professional manner the day to day running of the home’s administration, clinical policies and procedures, training and care planning. To implement working practices that monitors the health and welfare of the home’s service users and staff and their respective environments. To promote quality care within a warm friendly ambience. Key Result Areas Managing To work with the Directors to achieve the home’s financial targets. To manage the home in a manner which will not bring the home or service users into disrepute. To maintain confidentiality on all aspects of care and staff management. To ensure all the home’s policies and procedures are implemented and followed by all staff. To inform the Registered Manager immediately if a serious difficulty or event occurs. Managing Support To delegate responsibility effectively and within legal boundaries. To ensure through clinical standards and

#### A Few Statistics Before Any Further Pre-processing

In the following, we are interested to know a few statistics at this very begining stage, including:
* The total number of tokens across the corpus
* The total number of types across the corpus, i.e. the size of vocabulary 
* The so-called, [lexical diversity](https://en.wikipedia.org/wiki/Lexical_diversity), referring to the ratio of different unique word stems (types) to the total number of words (tokens).  
* The average, minimum and maximum number of token (i.e. document length) in the dataset.

In the following, we wrap all these up as a function, since we will use this printing module later to compare these statistic values before and after pre-processing.

In [12]:
def stats_print(tk_description):
    words = list(chain.from_iterable(tk_description)) # we put all the tokens in the corpus in a single list
    vocab = set(words) # compute the vocabulary by converting the list of words/tokens to a set, i.e., giving a set of unique words
    lexical_diversity = len(vocab)/len(words)
    print("Vocabulary size: ",len(vocab))
    print("Total number of tokens: ", len(words))
    print("Lexical diversity: ", lexical_diversity)
    print("Total number of description:", len(tk_description))
    lens = [len(article) for article in tk_description]
    print("Average description length:", np.mean(lens))
    print("Maximum description length:", np.max(lens))
    print("Minimum description length:", np.min(lens))
    print("Standard deviation of description length:", np.std(lens))

stats_print(tk_description)

Vocabulary size:  9834
Total number of tokens:  186952
Lexical diversity:  0.052601737344345076
Total number of description: 776
Average description length: 240.91752577319588
Maximum description length: 815
Minimum description length: 13
Standard deviation of description length: 124.97750685071483


## Task 1.2.4 Remove words with length less than 2.
+ We have 2 types of most frequent words is that in terms either term frequency or document frequency
+ In this sub-task, you are required to remove any token that only contains a single character (a token that of length less than 2).

In [13]:
words = list(chain.from_iterable(tk_description)) # we put all the tokens in the corpus in a single list
word_counts = Counter(words) # count the number of times each word appears in the corpus
print("Before removing, the number of words that appear with length less than 2:", len([w for w in word_counts if word_counts[w] < 2]))

# filter out single character tokens
tk_description = [[w for w in description if len(w) >=2] \
                      for description in tk_description]

words = list(chain.from_iterable(tk_description)) # we put all the tokens in the corpus in a single list
word_counts = Counter(words) # count the number of times each word appears in the corpus
print("After removing, the number of words that appear less than 2:", len([w for w in word_counts if word_counts[w] < 2]))

Before removing, the number of words that appear with length less than 2: 4233
After removing, the number of words that appear less than 2: 4231


### Task 2.3 Remove the top 50 most frequent words based on document frequency.
+ We have 2 types of most frequent words is that in terms either term frequency or document frequency

Explore the most frequent words in the pre-processed tokenized review text corpus:
* explore the most frequent words (top 50) based on term frequency and document frequency, respectively
* compare the results using different frequency measurements, which words are extracted based on both frequency measurements?
* think and decide on whether or not you would remove some of the most frequent words 

In [14]:
# Remove the top 50 most frequent words
words = list(chain.from_iterable(tk_description)) # we put all the tokens in the corpus in a single list
word_counts = Counter(words) # count the number of times each word appears in the corpus
top50 = word_counts.most_common(50) # get the top 50 most frequent words
print("Top 50 most frequent words:\n",top50, "\n\n")

tk_description = [[w for w in tk_description if w not in top50] for description in tk_description]
top50 = word_counts.most_common(50) # get the top 50 most frequent words
print("Top 50 most frequent words after removing:\n",top50)

Top 50 most frequent words:
 [('and', 8309), ('the', 6487), ('to', 6265), ('of', 4630), ('in', 3290), ('for', 2832), ('with', 2306), ('will', 2021), ('you', 2011), ('be', 1869), ('is', 1793), ('as', 1425), ('this', 1393), ('an', 1361), ('are', 1332), ('experience', 1276), ('on', 1216), ('have', 1114), ('or', 1088), ('sales', 1030), ('role', 946), ('work', 861), ('business', 832), ('your', 816), ('team', 789), ('we', 724), ('working', 719), ('our', 714), ('job', 688), ('care', 675), ('within', 672), ('skills', 669), ('all', 663), ('that', 625), ('company', 614), ('client', 594), ('their', 587), ('management', 572), ('manager', 519), ('from', 514), ('please', 504), ('support', 501), ('uk', 496), ('at', 491), ('service', 481), ('looking', 474), ('excellent', 455), ('development', 431), ('new', 429), ('must', 413)] 


Top 50 most frequent words after removing:
 [('and', 8309), ('the', 6487), ('to', 6265), ('of', 4630), ('in', 3290), ('for', 2832), ('with', 2306), ('will', 2021), ('you', 20

## Task 1.2.5 Remove stopwords using the provided stop words list (i.e, stopwords_en.txt)
> **Be CAREFUL**: as mentioned before, the purpose of this task is to pre-process the text reviews and later on we are going to use the pre-process text to build a sentiment analysis model. The stop word removal process requires careful consideration in this type of task.

Remove the stop words from the tokenized text inside `stopwords_en.txt` file

In [15]:
# remove the stop words inside `stopwords_en.txt` from the tokenized text
stopwords_file = 'stopwords_en.txt'

# read the stop words into a list
with open(stopwords_file, 'r') as f:
    stop_words = f.read().splitlines() 
print(f'The number of stop words inside {stopwords_file} is {len(stop_words)} including:\n\n{stop_words}')

The number of stop words inside stopwords_en.txt is 571 including:

['a', "a's", 'able', 'about', 'above', 'according', 'accordingly', 'across', 'actually', 'after', 'afterwards', 'again', 'against', "ain't", 'all', 'allow', 'allows', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'an', 'and', 'another', 'any', 'anybody', 'anyhow', 'anyone', 'anything', 'anyway', 'anyways', 'anywhere', 'apart', 'appear', 'appreciate', 'appropriate', 'are', "aren't", 'around', 'as', 'aside', 'ask', 'asking', 'associated', 'at', 'available', 'away', 'awfully', 'b', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'behind', 'being', 'believe', 'below', 'beside', 'besides', 'best', 'better', 'between', 'beyond', 'both', 'brief', 'but', 'by', 'c', "c'mon", "c's", 'came', 'can', "can't", 'cannot', 'cant', 'cause', 'causes', 'certain', 'certainly', 'changes', 'clearly', 'co', 'com', 'come', 'comes', 'concerning', 'conse

### -----------> OBSERVATION:
+ There 571 stopwords in total, which are often function words in English, like articles (e.g. "the", and "an"), pronouns (e.g. "he", "him", and "they"), particles (e.g., "well", "however" and "thus"), etc, and universal words in all job advertisement (e.g.'ask', 'asking', 'used', and 'useful')

+ Note this this is just one of the lists and we emphasize that there is no universal list of stop words. 

In [16]:
# flitering stop words
print("Before stopword removal:",len(tk_description)," tokens")
tk_description = [token for token in tk_description if token not in stop_words]
print("After stopword removal:",len(tk_description)," tokens")

Before stopword removal: 776  tokens
After stopword removal: 776  tokens


In [17]:
filtered_tokens = [token for token in tk_description if token in stop_words]
filtered_tokens

[]

In [18]:
[w for w in stop_words if ("not" in w or "n't" in w or "no" in w)]

["ain't",
 'another',
 "aren't",
 "can't",
 'cannot',
 "couldn't",
 "didn't",
 "doesn't",
 "don't",
 'enough',
 "hadn't",
 "hasn't",
 "haven't",
 'ignored',
 "isn't",
 'know',
 'knows',
 'known',
 'no',
 'nobody',
 'non',
 'none',
 'noone',
 'nor',
 'normally',
 'not',
 'nothing',
 'novel',
 'now',
 'nowhere',
 "shouldn't",
 "wasn't",
 "weren't",
 "won't",
 "wouldn't"]

## Task 1.2.6 Remove the word that appears only once in the document collection, based on term frequency

move on to the less frequent words:
* find out the list of words that appear only once in the **entire corpus**
* remove these less frequent words from each tokenized description text

We first need to find out the set of less frequent words by using the `hapaxes` function applied on the **term frequency** dictionary. 

In [19]:
from nltk.probability import *
from itertools import chain

words = list(chain.from_iterable(tk_description)) # we put all the tokens in the corpus in a single list
term_fd = FreqDist(words) # compute term frequency for each unique word/type

TypeError: unhashable type: 'list'

In [ ]:
lessFreqWords = set(term_fd.hapaxes())
print(len(lessFreqWords))
lessFreqWords

In [ ]:
def removeLessFreqWords(description):
    return [w for w in description if w not in lessFreqWords]

tk_reviews = [removeLessFreqWords(description) for description in tk_description]

stats_print(tk_reviews)

## Task 1.2.7 Remove the top 50 most frequent words based on document frequency.

In [ ]:
words = list(chain.from_iterable([set(description) for description in tk_description]))
doc_fd = FreqDist(words)  # compute document frequency for each unique word/type
top50MostFreqWords = doc_fd.most_common(50)
top50MostFreqWords

In [ ]:
def removeMostFreqWords(description):
    return [w for w in description if w not in top50MostFreqWords]

tk_reviews = [removeMostFreqWords(description) for description in tk_description]

stats_print(tk_reviews)

#### The Updated Statistics

In the above, we have done a few pre-processed steps, now let's have a look at the statistics again:


In [ ]:
# specify
ignored_words = [w for w in stop_words if not ("not" in w or "n't" in w or "no" in w)]

# filter out stop words
tk_description = [[w for w in description if w not in ignored_words] \
                      for description in tk_description]

stats_print(tk_description)

Recall, from the beginning, we have the following:  
_____________________________________________

Vocabulary size:  9834

Total number of tokens:  186952

Lexical diversity:  0.052601737344345076

Total number of description: 776

Average description length: 240.91752577319588

Maximun description length: 815

Minimun description length: 13

Standard deviation of description length: 124.97750685071483
_____________________________________________

We've shrunk more than 40% of the vocabulary.

category

## Task 1.2.8 Save all job advertisement text and information in txt files (we will retrieve them. in task 2 and 3)
Save the vocabulary, bigrams and job advertisment txt as per spectification.

We are going to store all the preprocessed description texts and its corresponding labels into files for task 2.
* all the tokenized description are stored in a .txt file named `description.txt`
    * each line is a description text, which contained all the tokens of the description text, separated by a space ' '
* all the corresponding labels are store in a .txt file named `category.txt`
    * each line is a label (one of these 4 values: 0,1,2,3)

In [ ]:
# save description text
def save_description(descriptionFilename,tk_description):
    out_file = open(descriptionFilename, 'w') # creates a txt file and open to save the descriptions
    string = "\n".join([" ".join(description) for description in tk_description])
    out_file.write(string)
    out_file.close() # close the file

# save the category corresponding with the description text
def save_category(categoryFilename,category):
    out_file = open(categoryFilename, 'w') # creates a txt file and open to save category
    string = "\n".join([str(s) for s in category])
    out_file.write(string)
    out_file.close() # close the file

# save the title corresponding with the description text
def save_title(titleFilename,title):
    out_file = open(titleFilename, 'w') # creates a txt file and open to save title
    string = "\n".join([str(s) for s in title])
    out_file.write(string)
    out_file.close() # close the file

# save description into txt file
descriptionFilename = "description.txt"
save_description(descriptionFilename,tk_description)
print(f'Successfully saved description into {descriptionFilename}')

# save category into txt file
categoryFilename = "category.txt"
save_category(categoryFilename,category)
print(f'Successfully saved category into {categoryFilename}')

# save title into txt file
titleFilename = "title.txt"
save_title(titleFilename,title)
print(f'Successfully saved title into {titleFilename}')

## Task 1.2.9 Build a vocabulary of the cleaned job advertisement descriptions

`vocab.txt`
This file contains the unigram vocabulary, one each line, in the following format: word_string:word_integer_index. Very importantly, words in the vocabulary must be sorted in alphabetical order, and the index value starts from 0. This file is the key to interpret the sparse encoding. For instance, in the following example, the word aaron is the 20th word (the corresponding integer_index as 19) in the vocabulary (note that the index values and words in the following image are artificial and used to demonstrate the required format only, it doesn't reflect the values of the actual expected output).

In [ ]:
# Save all job advertisement text and information in txt file
with open('job_ad.txt', 'w') as f:
    f.write("Category: " + str(category) + "\n")
    for i in range(len(tk_description)):
        f.write(full_description[i] + "\n")
        f.write("Tokenized Description: " + str(tk_description[i]) + "\n")
        f.write("Category: " + str(df['target'][i]) + "\n")
        f.write("\n")
    print("Successfully write job advertisement with the tokenized description in txt file")

In [ ]:
def write_vocab(vocab, filename):
    with open(filename, 'w') as f:
        for i, word in enumerate(vocab):
            f.write(word + ':' + str(i) + '\n')
# convert tokenized description into a alphabetically sorted list
vocab = sorted(list(set(chain.from_iterable(tk_description))))
write_vocab(vocab, 'vocab.txt')
# print out the first 10 words in the vocabulary
print(vocab[:10])

In [ ]:
print(f'Category at index 0: {df["target_names"][0]}')
print(f'Category at index 1: {df["target_names"][1]}')
print(f'Category at index 2: {df["target_names"][2]}')
print(f'Category at index 3: {df["target_names"][3]}')

In [ ]:
# convert job ad to a dataframe
job_ad = pd.DataFrame({'Title': title, 'Webindex': webindex, 'Company': company, 'Description': description,'Tokenized Description': tk_description, 'Category': category})

# change Tokenized Description to string separated by space
job_ad['Tokenized Description'] = job_ad['Tokenized Description'].apply(lambda x: ' '.join([str(i) for i in x]))


# replace the value in Category column
# Category at index 0: Accounting_Finance
# Category at index 1: Engineering
# Category at index 2: Healthcare_Nursing
# Category at index 3: Sales
job_ad['Category'] = job_ad['Category'].replace([0,1,2,3],['Accounting_Finance','Engineering','Healthcare_Nursing','Sales'])

# Cast Webindex to int
job_ad['Webindex'] = job_ad['Webindex'].astype(int)

# save job ad to csv file
job_ad.to_csv('job_ad.csv', index=False)

print(job_ad.info())
# print first 3 rows
job_ad.head(3)

In [20]:
# The .py format of the jupyter notebook
for fname in os.listdir():
    if fname.endswith('ipynb'):
        os.system(f'jupyter nbconvert {fname} --to python')

[NbConvertApp] Converting notebook task1.ipynb to python
[NbConvertApp] Writing 29547 bytes to task1.py
[NbConvertApp] Converting notebook task2_3.ipynb to python
[NbConvertApp] Writing 18131 bytes to task2_3.py


<h3 style="color:#ffc0cb;font-size:50px;font-family:Georgia;text-align:center;"><strong>1.3 Summary</strong></h3>

In this activity, we have demonstrated the basic text pre-processing steps of sentence segmentation and tokenization. 
There are a couple of things that we should keep in mind:

* we have covered the fundamentals of text pre-processing steps of Case Normalization, Stop Word Removing, Stemming and Lemmatization. 

* As mentioned before, though these steps are doing very different things to the text we have, however, one common effect among them, is the reduction on the size of the vocabulary (the list of distinct words contained in the corpus). 

* How we should process the text depends on the downstream analysis. Before we do any pre-processing, we should decide on the scope of the text to be used in the downstream analysis task. For instance, should we use an entire document? Or should we break the document down into sections, paragraphs, or sentences. Take another example. If we are analysing emails, should we keep the headers information? or should we focus on the email body? Choosing the proper scope depends on the goals of the analysis task. For example, you might choose to use an entire document in document classification and clustering tasks while one might choose smaller units like paragraphs or sentences in document summarization and information retrieval tasks. The scope chosen will have an  impact on the steps needed in the pre-processing process.

* In this activity, we have shown you multiple ways to do tokenization. However, there is no single right way to do tokenization.  It completely depends on the corpus and the text analysis task you are going to perform. The major question of the tokenization phase is what counts as a token. In some of the text analysis task. Although word tokenization is relatively easy compared with other NLP or text mining task, errors made in this phase will propagate into later analysis and cause problems.

* Case Normalization is a very simple process to do, though it is indeed very effective. 
The above is a very simple example (consisting of a very short paragraph) and it might not show much reduction on the vocabulary size. Imagine if you have a large corpus, doing case normalization will significantly reduce the vocabulary size, and thus helps the analysis algorithms to focus on different meaning of tokens rather than its cases.

In [ ]:
print("Number of distinct words BEFORE further pre-processing:",len(set(description)))
print("Number of distinct words AFTER further pre-processing:",len(set(tk_description)))

> # Discussion
>>In some of the text analysis tasks, we have to be mindful in the process of stopword removal. 
In some scenarios, stop words removal can wipe out relevant information and modify the context in a given sentence. 
For example, if we are performing a sentiment analysis, the word 'not', although is a stop word, it carries critical information, i.e. 'like' and 'not like' obviously are carrying completely reversed meaning.
We might fool out our algorithm off track if we remove a stop word like “not”. 
You should always carefully consider these conditions, design the list of "stop words" that are to removed based on your specific objectives.

<h3 style="color:#ffc0cb;font-size:50px;font-family:Georgia;text-align:center;"><strong>1.4 References</strong></h3>


+ [1] Sentence boundary disambiguation. https://en.wikipedia.org/wiki/Sentence_boundary_disambiguation
+ [2] Tokenization. https://nlp.stanford.edu/IR-book/html/htmledition/tokenization-1.html  
+ [3] Your Guide to Natural Language Processing (NLP). https://towardsdatascience.com/your-guide-to-natural-language-processing-nlp-48ea2511f6e1  
+ [4] Introduction to Natural Language Processing for Text. https://towardsdatascience.com/introduction-to-natural-language-processing-for-text-df845750fb63  
+ [5] [Accessing Text Corpora and Lexical Resources](http://www.nltk.org/book/ch02.html): Chapter 2 of "Natural Language Processing with Python" By Steven Bird, Ewan Kelin & Edward Loper.  
+ [6]. [Corpus Readers](http://www.nltk.org/howto/corpus.html#tagged-corpora): An NLTK tutorial on accessing the contents of a diverse set of corpora.

+ [1] Stop words. https://en.wikipedia.org/wiki/Stop_word  
+ [3] Bird, Steven, Edward Loper and Ewan Klein (2009), [Natural Language Processing with Python](http://www.nltk.org/book/). O’Reilly Media Inc.  